In [ ]:
import pymongo
import sys
import json
import time
import urllib.parse
from datetime import datetime
from pymongo import MongoClient
import datetime

import itertools
from itertools import groupby
from operator import itemgetter

#Importing other python files
import import_ipynb
import Automated_process_Standalone
from Automated_process_Standalone import *

client = MongoClient('localhost', 27017) #Establishing connection with MongoDB 
client.admin.authenticate('DasSystem', 'DasSystem',mechanism ='SCRAM-SHA-1', source='admin')
db = client['DASMongoDB'] #Database name
collection = db['DAS.Grob21.Package_2'] #Collection name to get the ToolID

##### SPLITTING FRAMES #####
#Initialize the table_values with the table name which you want to split frame data 
table_values = [
        {
            "tab_name":"Prediction_Spindle1",
            "column_name":"TrendingSp1",
            "sub_tables":["TrendingSp1_Value1","TrendingSP1_Value2","TrendingSp1__Value3","TrendingSp1_Value4"] ## any table name to create in mongoDB
        },
        {
            "tab_name":"Prediction_Spindle2",
            "column_name":"TrendSp2",
            "sub_tables":["TrendingSp2_Value1","TrendingSp2_Value2","TrendingSp2__Value3","TrendingSp2_Value4"] ## any table name to create in mongoDB
        }
]
    
#Function to get formatted timestamp 
def Time(t):
    t_year = t['timeStamp']['year']
    t_month = t['timeStamp']['month']
    t_day = t['timeStamp']['day']
    t_hour = t['timeStamp']['hour']
    t_minute = t['timeStamp']['minute']
    t_second = t['timeStamp']['second']
    t_ms = t['timeStamp']['ms']
    t_us = t['timeStamp']['us']
    x_1 = ('{0}-{1}-{2} {3}:{4}:{5}.{6}'.format(t_year,t_month, t_day, t_hour, t_minute, t_second, t_ms))
    return x_1

def getDataFrame(frameData, dateObj,files):
    #Variables for creating tables 
    timestampStr = dateObj.strftime("%Y-%m-%d %H:%M:%S.%f")
    Value1ToAdd = { "DateTime": timestampStr, "DataItem": [] }
    Value2ToAdd = { "DateTime": timestampStr, "DataItem": [] }
    Value3ToAdd = { "DateTime": timestampStr, "DataItem": [] }
    Value4ToAdd = { "DateTime": timestampStr, "DataItem": [] }
    for val in frameData:
        timeJson = Time(val)
        for idx, oneData in enumerate(val['data']):
            data1 ={ "TimeStamp":"", "Data":"" }
            data2 ={ "TimeStamp":"", "Data":"" }
            data3 ={ "TimeStamp":"", "Data":"" }
            data4 ={ "TimeStamp":"", "Data":"" }
            
            #Checking data array in frames
            if idx == 0:
                data1["TimeStamp"]=timeJson
                data1["Data"]=oneData
                Value1ToAdd["DataItem"].append(data1)
            elif idx == 1:
                data2["TimeStamp"]=timeJson
                data2["Data"]=oneData
                Value2ToAdd["DataItem"].append(data2)
            elif idx == 2:
                data3["TimeStamp"]=timeJson
                data3["Data"]=oneData 
                Value3ToAdd["DataItem"].append(data3)
            else:
                data4["TimeStamp"]=timeJson
                data4["Data"]=oneData 
                Value4ToAdd["DataItem"].append(data4)
                
    #Creating new tables for each values in data array   
    if Value1ToAdd is not 0:
        db[files[0]].insert_one(Value1ToAdd)
    if Value2ToAdd is not 0:
        db[files[1]].insert_one(Value2ToAdd)
    if Value3ToAdd is not 0:
        db[files[2]].insert_one(Value3ToAdd)
    if Value4ToAdd is not 0:
        db[files[3]].insert_one(Value4ToAdd)
        
##### START SPLITTING IF THE COLLECTION WITHIN RANGE #####    
def start(range1, range2):
    
    #Loop runs upto each object in the collection
    for table in table_values:
        table_name = table["tab_name"]
        tab = db[table_name].find()
        objCount = 0
        for val in tab:
            if (range1 <= val['DateTime'] <= range2):         
                try:
                    #Getting json data from data string    
                    objCount +=1
                    j_Data= json.loads(val[table["column_name"]])
                    frameData = j_Data["frames"]
                    if len(frameData)is not 0: 
                     getDataFrame(frameData,val["DateTime"],table["sub_tables"])
                except Exception as e:
                    #Identifying the missing datas in DB collection
                    print("May be missing a data in Db collection object no:", objCount, file=sys.stderr)     

##### FILTERING THE CODE FOR THE REQUIRED DATASET #####
completeSet = []; #Stores the collection records
currentSet = []; #Groups in terms of workpieces
completeSubset = []; #Lists only those records between the two tools
Last_tool_timestamp = datetime(2020,4,6) #Datetime from which it starts checking in the database collection;
                                         #If required to start from beginning, assign with 0.
    
# Storing the complete collection records in a variable
for x in collection.find():
 if (x['DateTime'] > Last_tool_timestamp):
  completeSet.append(x);

# Grouping in terms of the workpiece numbers (Makes sets)
sorted_completeSet = sorted(completeSet, key=itemgetter('Workpiece_1'))
currentSet= [list(group) for key, group in itertools.groupby(completeSet, key=lambda x: x['Workpiece_1'])]

# Filtering records between these two tools
tool1 = 17010;
tool2 = 17050;

for i in range(0,len(currentSet)):
    for j in range(0,len(currentSet[i])):
      for k in range(j+1,len(currentSet[i])):
        if (currentSet[i][j]['Tool_ID'] == tool1 and currentSet[i][k]['Tool_ID'] == tool2):
         completeSubset.append([dict(currentSet[i][x])for x in range(j,k+1)])

# Splitting the frames for each group of workpiece_numbers            
for i in range(0,len(completeSubset)): 
    start(completeSubset[i][0]['DateTime'], completeSubset[i][len(completeSubset[i])-1]['DateTime'])
    for j in range(0,len(completeSubset[i])):
        Last_tool_timestamp = completeSubset[i][j]['DateTime']; 

        if (((i+1) == len(completeSubset)) and ((j+1) == len(completeSubset[i]))):
          Last_folder = Integrated_Code_1.start(completeSubset[0][0]['DateTime'],completeSubset[i][j]['DateTime']) #Calling the Integrated code
  
Automated_DataPrep.PrepareData(Last_folder); #Calling the feature extraction code
Automated_process_Standalone.Restart(Last_tool_timestamp); #Calling the standalone code that runs every Xmins   

importing Jupyter notebook from Automated_process_Standalone.ipynb
importing Jupyter notebook from Integrated_Code_1.ipynb
importing Jupyter notebook from Automated_DataPrep.ipynb


May be missing a data in Db collection object no: 52
May be missing a data in Db collection object no: 57
May be missing a data in Db collection object no: 63
May be missing a data in Db collection object no: 135
May be missing a data in Db collection object no: 138
May be missing a data in Db collection object no: 160
May be missing a data in Db collection object no: 189
May be missing a data in Db collection object no: 191
May be missing a data in Db collection object no: 213
May be missing a data in Db collection object no: 284
May be missing a data in Db collection object no: 291
May be missing a data in Db collection object no: 308
May be missing a data in Db collection object no: 399
May be missing a data in Db collection object no: 418
May be missing a data in Db collection object no: 419
May be missing a data in Db collection object no: 513
May be missing a data in Db collection object no: 575
May be missing a data in Db collection object no: 620
May be missing a data in Db col

May be missing a data in Db collection object no: 1396
May be missing a data in Db collection object no: 1429
May be missing a data in Db collection object no: 1440
May be missing a data in Db collection object no: 1516
May be missing a data in Db collection object no: 1591
May be missing a data in Db collection object no: 1653
May be missing a data in Db collection object no: 1777
May be missing a data in Db collection object no: 1791
May be missing a data in Db collection object no: 1810
May be missing a data in Db collection object no: 9
May be missing a data in Db collection object no: 13
May be missing a data in Db collection object no: 21
May be missing a data in Db collection object no: 36
May be missing a data in Db collection object no: 38
May be missing a data in Db collection object no: 80
May be missing a data in Db collection object no: 127
May be missing a data in Db collection object no: 129
May be missing a data in Db collection object no: 138
May be missing a data in D

May be missing a data in Db collection object no: 473
May be missing a data in Db collection object no: 481
May be missing a data in Db collection object no: 490
May be missing a data in Db collection object no: 497
May be missing a data in Db collection object no: 508
May be missing a data in Db collection object no: 549
May be missing a data in Db collection object no: 551
May be missing a data in Db collection object no: 603
May be missing a data in Db collection object no: 618
May be missing a data in Db collection object no: 635
May be missing a data in Db collection object no: 641
May be missing a data in Db collection object no: 644
May be missing a data in Db collection object no: 687
May be missing a data in Db collection object no: 691
May be missing a data in Db collection object no: 695
May be missing a data in Db collection object no: 716
May be missing a data in Db collection object no: 723
May be missing a data in Db collection object no: 751
May be missing a data in Db 

May be missing a data in Db collection object no: 696
May be missing a data in Db collection object no: 757
May be missing a data in Db collection object no: 792
May be missing a data in Db collection object no: 843
May be missing a data in Db collection object no: 847
May be missing a data in Db collection object no: 850
May be missing a data in Db collection object no: 884
May be missing a data in Db collection object no: 897
May be missing a data in Db collection object no: 900
May be missing a data in Db collection object no: 904
May be missing a data in Db collection object no: 995
May be missing a data in Db collection object no: 1086
May be missing a data in Db collection object no: 1088
May be missing a data in Db collection object no: 1160
May be missing a data in Db collection object no: 1173
May be missing a data in Db collection object no: 1238
May be missing a data in Db collection object no: 1282
May be missing a data in Db collection object no: 1299
May be missing a data

May be missing a data in Db collection object no: 1258
May be missing a data in Db collection object no: 1274
May be missing a data in Db collection object no: 1305
May be missing a data in Db collection object no: 1315
May be missing a data in Db collection object no: 1317
May be missing a data in Db collection object no: 1360
May be missing a data in Db collection object no: 1365
May be missing a data in Db collection object no: 1367
May be missing a data in Db collection object no: 1408
May be missing a data in Db collection object no: 1538
May be missing a data in Db collection object no: 1567
May be missing a data in Db collection object no: 1618
May be missing a data in Db collection object no: 1620
May be missing a data in Db collection object no: 1645
May be missing a data in Db collection object no: 1647
May be missing a data in Db collection object no: 1776
May be missing a data in Db collection object no: 1785
May be missing a data in Db collection object no: 45
May be missi

May be missing a data in Db collection object no: 611
May be missing a data in Db collection object no: 621
May be missing a data in Db collection object no: 632
May be missing a data in Db collection object no: 639
May be missing a data in Db collection object no: 671
May be missing a data in Db collection object no: 673
May be missing a data in Db collection object no: 675
May be missing a data in Db collection object no: 715
May be missing a data in Db collection object no: 717
May be missing a data in Db collection object no: 727
May be missing a data in Db collection object no: 735
May be missing a data in Db collection object no: 747
May be missing a data in Db collection object no: 755
May be missing a data in Db collection object no: 770
May be missing a data in Db collection object no: 787
May be missing a data in Db collection object no: 816
May be missing a data in Db collection object no: 843
May be missing a data in Db collection object no: 850
May be missing a data in Db 

May be missing a data in Db collection object no: 91
May be missing a data in Db collection object no: 144
May be missing a data in Db collection object no: 152
May be missing a data in Db collection object no: 184
May be missing a data in Db collection object no: 198
May be missing a data in Db collection object no: 208
May be missing a data in Db collection object no: 239
May be missing a data in Db collection object no: 252
May be missing a data in Db collection object no: 261
May be missing a data in Db collection object no: 299
May be missing a data in Db collection object no: 317
May be missing a data in Db collection object no: 338
May be missing a data in Db collection object no: 347
May be missing a data in Db collection object no: 377
May be missing a data in Db collection object no: 397
May be missing a data in Db collection object no: 414
May be missing a data in Db collection object no: 454
May be missing a data in Db collection object no: 470
May be missing a data in Db c

May be missing a data in Db collection object no: 351
May be missing a data in Db collection object no: 366
May be missing a data in Db collection object no: 377
May be missing a data in Db collection object no: 387
May be missing a data in Db collection object no: 399
May be missing a data in Db collection object no: 405
May be missing a data in Db collection object no: 414
May be missing a data in Db collection object no: 425
May be missing a data in Db collection object no: 429
May be missing a data in Db collection object no: 436
May be missing a data in Db collection object no: 453
May be missing a data in Db collection object no: 463
May be missing a data in Db collection object no: 467
May be missing a data in Db collection object no: 469
May be missing a data in Db collection object no: 471
May be missing a data in Db collection object no: 473
May be missing a data in Db collection object no: 478
May be missing a data in Db collection object no: 492
May be missing a data in Db 